# 코로나 데이터 분석

## 프로젝트 목표

* 서울시 코로나19 확진자 현황 데이터를 분석하여 유의미한 정보 도출
* 탐색적 데이터 분석을 수행하기 위한 데이터 정제, 특징 엔지니어링, 시각화 방법 학습

## 프로젝트 목차
1. **데이터 읽기:** 코로나 데이터를 불러오고 DataFrame 구조를 확인<br>
    1.1 데이터 불러오기<br>
<br>
2. **데이터 청제:** 비어 있는 데이터 또는 쓸모 없는 데이터를 삭제<br>
    2.1 비어있는 column 지우기<br>
<br>
3. **데이터 시각화:** 각 변수 별로 추가적인 정제 또는 feature engineering 과정을 거치고 시각화를 통하여 데이터의 특성 파악<br>
    3.1 확진일 데이터 전처리하기 <br>
    3.2 월별 확진자 수 출력 <br>
    3.3 8월 일별 확진자 수 출력 <br>
    3.4 지역별 확진자 수 출력 <br>
    3.5 8월달 지역별 확진자 수 출력 <br>
    3.6 월별 관악구 확진자 수 출력 <br>
    3.7 서울 지역에서 확진자를 지도에 출력 <br>
   

## 데이터 출처
 * https://www.data.go.kr/tcs/dss/selectFileDataDetailView.do?publicDataPk=15063273

---

## 프로젝트 개요

2020년 초에 발생한 코로나19바이러스는 세계적으로 대유행하였고 이에 대한 많은 분석이 이루어지고 있습니다. 유행 초기엔 이를 분석할 데이터가 충분하지 않았지만 6개월 이상이 지난 지금은 다양한 데이터 기관에서 코로나 관련 데이터를 공공으로 제공하고 있습니다.


이번 프로젝트에서는 국내 공공데이터 포털에서 제공하는 `서울시 코로나 19확진자 현황` 데이터를 바탕으로 탐색적 데이터 분석을 수행해보겠습니다. 국내 데이터 중 확진자 비율이 제일 높고 사람이 제일 많은 서울시의 데이터를 선정하였으며, 이를 바탕으로 코로나19의 확진 추이 및 환자 특성에 대해서 데이터를 바탕으로 알아봅시다.

---

### 1. 데이터 읽기


필요한 패키지 설치 및 `import`한 후 `pandas` 를 사용하여 데이터를 읽고 어떠한 데이터가 저장되어 있는지 확인합니다.


#### 1.1 데이터 불러오기

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
corona_all = pd.read_csv("서울시 코로나19 확진자 현황.csv")

In [7]:
corona_all.head()

,연번,확진일,환자번호,국적,환자정보,지역,여행력,접촉력,조치사항,상태,이동경로,등록일,수정일,노출여부
0,67422,2021-08-06,NaN,NaN,NaN,강서구,NaN,감염경로 조사중,NaN,-,NaN,2021-08-07 10:48:45,2021-08-07 10:48:45,Y
1,67421,2021-08-06,NaN,NaN,NaN,영등포구,NaN,기타 확진자 접촉,NaN,-,NaN,2021-08-07 10:48:45,2021-08-07 10:48:45,Y
2,67420,2021-08-06,NaN,NaN,NaN,중구,NaN,타시도 확진자 접촉,NaN,-,NaN,2021-08-07 10:48:45,2021-08-07 10:48:45,Y
3,67419,2021-08-06,NaN,NaN,NaN,송파구,NaN,감염경로 조사중,NaN,-,NaN,2021-08-07 10:48:45,2021-08-07 10:48:45,Y
4,67418,2021-08-06,NaN,NaN,NaN,은평구,NaN,기타 확진자 접촉,NaN,-,NaN,2021-08-07 10:48:45,2021-08-07 10:48:45,Y


In [8]:
corona_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67422 entries, 0 to 67421
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      67422 non-null  int64  
 1   확진일     67422 non-null  object 
 2   환자번호    0 non-null      float64
 3   국적      0 non-null      float64
 4   환자정보    0 non-null      float64
 5   지역      67422 non-null  object 
 6   여행력     1436 non-null   object 
 7   접촉력     67422 non-null  object 
 8   조치사항    0 non-null      float64
 9   상태      67422 non-null  object 
 10  이동경로    0 non-null      float64
 11  등록일     67422 non-null  object 
 12  수정일     67422 non-null  object 
 13  노출여부    67422 non-null  object 
dtypes: float64(5), int64(1), object(8)
memory usage: 7.2+ MB


---

### 2. 데이터 정제

데이터를 읽고 확인했다면 결측값(missing data), 이상치(outlier)를 처리하는 데이터 정제 과정을 수행하여 봅시다.

#### 2.1 비어있는 column 지우기

`corona_all.info()` 코드를 통하여 `국적`, `환자정보`, `조치사항`에 해당하는 데이터가 존재하지 않는 것을 알 수 있습니다.<br>
`dataframe.drop()`를 사용하여 불필요한 `국적`, `환자정보`, `조치사항`의 column 데이터를 삭제하고 이 dataframe을 `corona_del_col`에 저장해 봅시다.

In [9]:
corona_del_col = corona_all.drop(columns = ['국적', '환자정보', '조치사항'])

In [10]:
corona_del_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67422 entries, 0 to 67421
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      67422 non-null  int64  
 1   확진일     67422 non-null  object 
 2   환자번호    0 non-null      float64
 3   지역      67422 non-null  object 
 4   여행력     1436 non-null   object 
 5   접촉력     67422 non-null  object 
 6   상태      67422 non-null  object 
 7   이동경로    0 non-null      float64
 8   등록일     67422 non-null  object 
 9   수정일     67422 non-null  object 
 10  노출여부    67422 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 5.7+ MB


---

### 3. 데이터 시각화

데이터 정제를 완료한 `corona_del_col` 데이터를 바탕으로 각 column의 변수별로 어떠한 데이터 분포를 하고 있는지 시각화를 통하여 알아봅시다.

#### 3.1 확진일 데이터 전처리하기

`확진일` 데이터를 간단히 출력해보면 `월,일` 형태의 날짜 형식임을 알 수 있습니다. <br>
월별, 일별 분석을 위해서는 문자열 형식의 데이터를 나누어 숫자 형 데이터로 변환해 보겠습니다.

In [11]:
corona_del_col['확진일']

0        2021-08-06
1        2021-08-06
2        2021-08-06
3        2021-08-06
4        2021-08-06
            ...    
67417    2020-01-31
67418    2020-01-30
67419    2020-01-30
67420    2020-01-30
67421    2020-01-24
Name: 확진일, Length: 67422, dtype: object

`확진일` 데이터를 `month`, `day` 데이터로 나누기<br>
`확진일`에 저장된 문자열 데이터를 나누어 `month`, `day` column에 int64 형태로 저장해 봅시다.

In [15]:
month = []
day = []

for data in corona_del_col['확진일'] :
    month.append(data.split('-')[1])
    day.append(data.split('-')[2])